In [8]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython.display import Image


In [27]:
# بارگذاری داده‌ها
lfw = fetch_lfw_people(min_faces_per_person=100, resize=0.4)  # ابتدا با حداقل ۲۰ شروع کن

print("توزیع تصاویر بین افراد:")
unique, counts = np.unique(lfw.target, return_counts=True)
for idx, count in zip(unique, counts):
    print(f"{lfw.target_names[idx]}: {count} تصویر")

print(f"\nمجموع تصاویر: {lfw.data.shape[0]}")
print(f"تعداد افراد: {len(lfw.target_names)}")

توزیع تصاویر بین افراد:
Colin Powell: 236 تصویر
Donald Rumsfeld: 121 تصویر
George W Bush: 530 تصویر
Gerhard Schroeder: 109 تصویر
Tony Blair: 144 تصویر

مجموع تصاویر: 1140
تعداد افراد: 5
